In [20]:
import sklearn as sk
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split



In [43]:
#load the data
#show all columns
pd.set_option('display.max_columns', None)
df = pd.read_csv('big_data_added_columns.csv')
df.head()

/var/folders/kc/jdw9593x3ksdmnhyd571zm440000gp/T/ipykernel_21291/1900148927.py:4: DtypeWarning: Columns (2,5,6,8,10,11,14,15,16,17,18,19,20,21,23,24,25,27,28,32,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('big_data_added_columns.csv')


,ROW_NUM,USER_ID,INV_OPEN_DATE,INV_NUM_ACCOUNTS,INV_ACC_BAL,MONEY_ACCOUNT_OPEN_DATE,MONEY_ACCOUNT_STATUS,MONEY_FICO,MONEY_DATA_UPDATE_DATE,MONEY_TOTAL_BALANCE,LAST_PAYMENT_DATE,ORIGINATION_DATE,PRINCIPAL,RELAY_FICO,SL_ACTIVE_IND,PL_ACTIVE_IND,MORTGAGE_ACTIVE_IND,BANKING_ACTIVE_IND,INVEST_ACTIVE_IND,CC_ACTIVE_IND,CSM_ACTIVE_IND,ACTIVE,TOTAL_RECENCY_DAYS,RELAY_ORIGINAL_DATE,ONBOARDING_SUCCESS_DATE,FIRST_TRANSACTION_DATE,CREDIT_LINE,CURRENT_CYCLE_BEHAVIOR_SEGMENT,MQS_REPORTING_DATE,REL_MON_MQS,REL_INV_MQS,REL_LOAN_MQS,SQS_DATE_EST,REL_MON_SQS,REL_INV_SQS,REL_LOAN_SQS,uses_invest,uses_money,uses_relay,uses_loan,uses_credit,num_products,date_of_creating_new_columns,days_with_invest,days_with_money,days_with_relay,days_with_loan,days_with_credit,first_product,second_product
0,1.0,22477118.0,NaN,NaN,NaN,2022-02-18,Active,850.0,2023-03-21,32148.42,NaN,NaN,NaN,850.0,False,False,False,True,False,False,True,True,1241.0,2019-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,2,2023-03-31,NaN,406.0,1249.0,NaN,NaN,relay,money
1,1.0,22477129.0,2022-02-19,2.0,65.40,2022-02-18,Active,9003.0,2023-03-21,0.50,NaN,NaN,NaN,NaN,False,False,False,True,True,False,True,True,113.0,2022-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,False,False,3,2023-03-31,405.0,406.0,121.0,NaN,NaN,money,invest
2,1.0,22477145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,932.0,2020-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False,False,1,2023-03-31,NaN,NaN,940.0,NaN,NaN,relay,NaN
3,1.0,22477153.0,NaN,NaN,NaN,2022-02-19,Active,556.0,2023-03-21,0.93,NaN,NaN,NaN,556.0,False,False,False,True,False,False,True,True,231.0,2022-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,False,False,2,2023-03-31,NaN,405.0,239.0,NaN,NaN,money,relay
4,1.0,22477154.0,2022-02-18,3.0,257713.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,662.0,False,False,False,False,True,False,True,True,128.0,2022-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,False,False,2,2023-03-31,406.0,NaN,136.0,NaN,NaN,invest,relay


In [49]:
df.value_counts('second_product')

second_product
relay     450801
invest    265788
money     198001
loan       47399
credit     41699
dtype: int64

# predicting uses_money w/ days with other products

In [30]:
#keep only the columns 'days_with_invest', 'days_with_relay', 'days_with_loan', 'days_with_credit', 'uses_money'
data = df[['days_with_invest', 'days_with_relay', 'days_with_loan', 'days_with_credit', 'uses_money', 'num_products', 'first_product', 'second_product']]

#replace nan with 0 (no days with that product)
data = data.fillna(0)

#downsample the data to balance the response variable (uses_money) (majority are users that do not use money and only use 1 product)

# Separate majority class (uses_money = 0 and num_products = 1)
data_majority = data[(data.uses_money == 0) & (data.num_products == 1)]
data_minority = data[(data.second_product == 'money') & (data.num_products == 2)]

# Downsample majority class
data_majority_downsampled = resample(data_majority, 
                                    replace=False,    # sample without replacement
                                    n_samples=len(data_minority),     # to match minority class
                                    random_state=123) # reproducible results

# Combine minority class with downsampled majority class
data_downsampled = pd.concat([data_majority_downsampled, data_minority])

# Display new class counts
data_downsampled.uses_money.value_counts()

#remove the columns 'num_products', 'first_product', 'second_product'
data_downsampled = data_downsampled[['days_with_invest', 'days_with_relay', 'days_with_loan', 'days_with_credit', 'uses_money']]
#shuffle the data 
data_downsampled = data_downsampled.sample(frac=1).reset_index(drop=True)

data_downsampled.head(20)


,days_with_invest,days_with_relay,days_with_loan,days_with_credit,uses_money
0,0.0,1055.0,0.0,0.0,True
1,0.0,128.0,0.0,0.0,False
2,425.0,0.0,0.0,0.0,True
3,0.0,0.0,256.0,0.0,True
4,0.0,980.0,0.0,0.0,False
5,0.0,107.0,0.0,0.0,False
6,679.0,0.0,0.0,0.0,False
7,700.0,0.0,0.0,0.0,False
8,0.0,169.0,0.0,0.0,True
9,0.0,264.0,0.0,0.0,True


In [31]:
#split the data into training and testing sets
X = data_downsampled.drop('uses_money', axis=1)
y = data_downsampled['uses_money']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#train the model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

#predict the response variable for the test set
y_pred = logreg.predict(X_test)

#evaluate the model
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))



0.6520433811208342


In [32]:
print(logreg.intercept_)
print(logreg.coef_)
#show which features are most important, include the column names
print(list(zip(X_train, logreg.coef_[0])))
print(list(zip(X_train, np.exp(logreg.coef_[0]))))


[-0.13715641]
[[-0.00011736  0.00063479 -0.00188442  0.0002882 ]]
[('days_with_invest', -0.00011735908611211262), ('days_with_relay', 0.0006347865664522207), ('days_with_loan', -0.001884421258557224), ('days_with_credit', 0.0002882046895992275)]
[('days_with_invest', 0.9998826478001961), ('days_with_relay', 1.0006349880860832), ('days_with_loan', 0.9981173531484309), ('days_with_credit', 1.0002882462245608)]


# predicting uses_loan w/ days with other products

In [33]:
# keep only the columns 'days_with_invest', 'days_with_relay', 'days_with_money', 'days_with_credit', 'uses_loan'
data = df[['days_with_invest', 'days_with_relay', 'days_with_money', 'days_with_credit', 'uses_loan', 'num_products', 'first_product', 'second_product']]
#replace nan with 0 (no days with that product)
data = data.fillna(0)
#downsample the data to balance the response variable (uses_loan) (majority are users that do not use loan and only use 1 product)

# Separate majority class (uses_loan = 0 and num_products = 1)
data_majority = data[(data.uses_loan == 0) & (data.num_products == 1)]
data_minority = data[(data.second_product == 'loan') & (data.num_products == 2)]

# Downsample majority class
data_majority_downsampled = resample(data_majority,
                                    replace=False,    # sample without replacement
                                    n_samples=len(data_minority),     # to match minority class
                                    random_state=123) # reproducible results

# Combine minority class with downsampled majority class
data_downsampled = pd.concat([data_majority_downsampled, data_minority])

# Display new class counts
data_downsampled.uses_loan.value_counts()

#remove the columns 'num_products', 'first_product', 'second_product'
data_downsampled = data_downsampled[['days_with_invest', 'days_with_relay', 'days_with_money', 'days_with_credit', 'uses_loan']]
#shuffle the data
data_downsampled = data_downsampled.sample(frac=1).reset_index(drop=True)

data_downsampled.head(20)


,days_with_invest,days_with_relay,days_with_money,days_with_credit,uses_loan
0,0.0,203.0,0.0,0.0,False
1,0.0,316.0,0.0,0.0,True
2,0.0,0.0,0.0,582.0,True
3,0.0,315.0,0.0,0.0,False
4,0.0,366.0,0.0,0.0,False
5,716.0,0.0,0.0,0.0,True
6,0.0,0.0,369.0,0.0,False
7,0.0,359.0,0.0,0.0,False
8,0.0,319.0,0.0,0.0,False
9,0.0,0.0,0.0,383.0,True


In [34]:
#split the data into training and testing sets
X = data_downsampled.drop('uses_loan', axis=1)
y = data_downsampled['uses_loan']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#train the model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

#predict the response variable for the test set
y_pred = logreg.predict(X_test)

#evaluate the model
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

print(logreg.intercept_)
#show which features are most important, include the column names
print(list(zip(X_train, logreg.coef_[0])))
print(list(zip(X_train, np.exp(logreg.coef_[0]))))



0.611746847267632
[-0.25368819]
[('days_with_invest', 0.0005156623919340901), ('days_with_relay', -0.00027242637990311107), ('days_with_money', 0.0011948901694648745), ('days_with_credit', 0.0026431046497650606)]
[('days_with_invest', 1.0005157953686414), ('days_with_relay', 0.9997276107247937), ('days_with_money', 1.0011956043351449), ('days_with_credit', 1.0026466007303503)]


# Revisions 

- goal: Identify the average time between first_product adoption and second_product adoption for each product pair




# notes
- difference between users that crossbuy and those that don't? credit score? etc.? 
- SQS?
- Michael knows about MQS data

# WHEN DO USERS CROSSBUY, WHAT DOES THAT PATH LOOK LIKE? 
- What more can we use to predict crossbuying - like interest rate hikes? Time of year? Seasonality? SoFi announcements? 

# IS THERE A DIFFERENCE BETWEEN USERS THAT CROSSBUY AND USERS THAT DONT?



In [15]:
time_to_crossbuy = data.copy()

In [16]:
first_product_relay = time_to_crossbuy[time_to_crossbuy.first_product == 'relay']
first_product_relay.value_counts('second_product')

second_product
money     117421
invest     56315
loan        7777
credit       844
dtype: int64

In [19]:
time_to_crossbuy.value_counts('second_product')

second_product
relay     444035
invest    264803
money     196551
loan       46923
credit     40303
dtype: int64

In [ ]:
#plot the number of people adopting a second product over time
#histograms of time to xbuy for different years
